In [1]:
!pip install muon
!pip install scanpy
!pip install scirpy
!pip install cycler

  Using cached umap_learn-0.5.9.post2-py3-none-any.whl.metadata (25 kB)
  Using cached protobuf-6.32.0-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached legacy_api_wrap-1.4.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached natsort-8.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 49.5 MB/s eta 0:00:00
Using cached legacy_api_wrap-1.4.1-py3-none-any.whl (10.0 kB)
Using cached natsort-8.4.0-py3-none-any.whl (38 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 51.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 50.0 MB/s eta 0:00

In [1]:
import muon as mu
import numpy as np
import scanpy as sc
import scirpy as ir
from cycler import cycler
from matplotlib import cm as mpl_cm
from matplotlib import pyplot as plt

# temporary fix for deprecated matplotlib functionality
import IPython.display
from matplotlib_inline.backend_inline import set_matplotlib_formats

IPython.display.set_matplotlib_formats = set_matplotlib_formats

sc.set_figure_params(figsize=(4, 4))
sc.settings.verbosity = 2  # verbosity: errors (0), warnings (1), info (2), hints (3)

/opt/conda/envs/scTCR/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mdata = ir.datasets.wu2020()
print(mdata)

/opt/conda/envs/scTCR/lib/python3.12/site-packages/anndata/utils.py:362: ExperimentalFeatureWarning: Support for Awkward Arrays is currently experimental. Behavior may change in the future. Please report any issues you may encounter!
  warnings.warn(msg, category, stacklevel=stacklevel)


MuData object with n_obs × n_vars = 161015 × 30727
  2 modalities
    gex:	141623 x 30727
      obs:	'cluster_orig', 'patient', 'sample', 'source'
      uns:	'cluster_orig_colors'
      obsm:	'X_umap_orig'
    airr:	123134 x 0
      obs:	'high_confidence', 'is_cell', 'clonotype_orig'
      obsm:	'airr'


/opt/conda/envs/scTCR/lib/python3.12/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/opt/conda/envs/scTCR/lib/python3.12/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [1]:
sc.pp.filter_genes(mdata["gex"], min_cells=10)
sc.pp.filter_cells(mdata["gex"], min_genes=100)

NameError: name 'sc' is not defined

In [8]:
sc.pp.normalize_per_cell(mdata["gex"])
sc.pp.log1p(mdata["gex"])
sc.pp.highly_variable_genes(mdata["gex"], flavor="cell_ranger", n_top_genes=5000)
sc.tl.pca(mdata["gex"])
sc.pp.neighbors(mdata["gex"])

normalizing by total count per cell


/tmp/ipykernel_9679/3237590329.py:1: FutureWarning: Use sc.pp.normalize_total instead
  sc.pp.normalize_per_cell(mdata["gex"])


    finished (0:00:00): normalized adata.X and added
    'n_counts', counts per cell before normalization (adata.obs)


/usr/local/python/3.12.1/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:591: FutureWarning: Use sc.pp.normalize_total instead
  normalize_per_cell(


extracting highly variable genes
    finished (0:00:03)
computing PCA
    with n_comps=50
    finished (0:00:10)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished (0:00:49)


In [16]:
ir.pp.index_chains(mdata)

Filtering chains...
Indexing VJ chains...
Indexing VDJ chains...
build result array


In [17]:
ir.tl.chain_qc(mdata)

Stored result in `mdata.obs["airr:receptor_type"]`.
Stored result in `mdata.obs["airr:receptor_subtype"]`.
Stored result in `mdata.obs["airr:chain_pairing"]`.


In [19]:
mu.pp.filter_obs(mdata, "airr:chain_pairing", lambda x: x != "multichain")
mu.pp.filter_obs(mdata, "airr:chain_pairing", lambda x: ~np.isin(x, ["orphan VDJ", "orphan VJ"]))

In [21]:
# using default parameters, `ir_dist` will compute nucleotide sequence identity
ir.pp.ir_dist(mdata)
ir.tl.define_clonotypes(mdata, receptor_arms="all", dual_ir="primary_only")
mdata.obs.groupby("gex:source", dropna=False).size()

Computing sequence x sequence distance matrix for VJ sequences.
Computing sequence x sequence distance matrix for VDJ sequences.
Initializing lookup tables. 
Computing clonotype x clonotype distances.
Stored result in `mdata.obs["airr:clone_id"]`.
Stored result in `mdata.obs["airr:clone_id_size"]`.


/tmp/ipykernel_9679/1251107014.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mdata.obs.groupby("gex:source", dropna=False).size()


gex:source
Blood     6243
NAT      46064
Tumor    52784
NaN      14617
dtype: int64

In [ ]:
print(mdata)

NameError: name 'mdata' is not defined

In [13]:
mdata.update()
# Get metadata from both modalities
gex_obs = mdata.mod["gex"].obs
airr_obs = mdata.mod["airr"].obs

/usr/local/python/3.12.1/lib/python3.12/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/usr/local/python/3.12.1/lib/python3.12/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [14]:
gex_obs

,cluster_orig,patient,sample,source,n_genes,n_counts
RT2_AAACCTGAGGTAGCCA-1,8.2-Tem,Renal2,RT2,Tumor,1753,5036.0
RT2_AAACCTGCAGGTCCAC-1,8.3b-Trm,Renal2,RT2,Tumor,1077,2512.0
RT2_AAACCTGCATCCGTGG-1,8.3c-Trm,Renal2,RT2,Tumor,1338,3237.0
RT2_AAACCTGCATCGTCGG-1,4.3-TCF7,Renal2,RT2,Tumor,1351,4250.0
RT2_AAACCTGGTGCCTGTG-1,4.2-RPL32,Renal2,RT2,Tumor,799,1936.0
...,...,...,...,...,...,...
CT2_TTTGCGCCACCGGAAA-1,8.3a-Trm,Colon2,CT2,Tumor,1171,3134.0
CT2_TTTGGTTAGCCACGCT-1,8.3c-Trm,Colon2,CT2,Tumor,1424,3483.0
CT2_TTTGGTTGTGCCTTGG-1,4.4-FOS,Colon2,CT2,Tumor,741,1421.0
CT2_TTTGGTTTCACTTACT-1,4.3-TCF7,Colon2,CT2,Tumor,1301,3528.0


In [15]:
airr_obs

,high_confidence,is_cell,clonotype_orig
RT2_AAACCTGAGGTAGCCA-1,True,True,renal2.tnb.C1
RT2_AAACCTGCAGGTCCAC-1,True,True,renal2.tnb.C2
RT2_AAACCTGCATCCGTGG-1,True,True,renal2.tnb.C1
RT2_AAACCTGCATCGTCGG-1,True,True,renal2.tnb.C3
RT2_AAACCTGTCTGAGTGT-1,True,True,renal2.tnb.C4
...,...,...,...
CT2_TTTGGTTGTAGCTAAA-1,True,True,NaN
CT2_TTTGGTTGTGCCTTGG-1,True,True,colon2.tn.C954
CT2_TTTGGTTTCACTTACT-1,True,True,colon2.tn.C955
CT2_TTTGGTTTCAGCTTAG-1,True,True,NaN
